In [1]:
pwd

'/home/wsuser/work'

In [2]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='FEkZJcucevmOjIfQuEuc3Y7oZyjw6Eo5ROxfgfqOilPQ',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'naturaldisasterintensityanalysisa-donotdelete-pr-pu6qhf6neteqe2'
object_key = 'dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [4]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [5]:
 train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,vertical_flip=True)

In [6]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [7]:
x_train=train_datagen.flow_from_directory(r"/home/wsuser/work/dataset/train_set",target_size=(64,64),batch_size=32,class_mode="categorical")

Found 742 images belonging to 4 classes.


In [8]:
x_test=train_datagen.flow_from_directory(r"/home/wsuser/work/dataset/test_set",target_size=(64,64),batch_size=32,class_mode="categorical")

Found 198 images belonging to 4 classes.


In [9]:
x_train.class_indices

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}

In [10]:
x_test.class_indices

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}

In [11]:
model=Sequential()

In [12]:
model.add(Convolution2D(32,(3,3),input_shape=(64,64,3),activation='relu'))

In [13]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [14]:
model.add(Flatten())

In [15]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [16]:
model.add(Dense(units=300,kernel_initializer="random_uniform",activation='relu'))

In [17]:
model.add(Dense(units=200,kernel_initializer="random_uniform",activation='relu'))

In [18]:
model.add(Dense(units=4,kernel_initializer="random_uniform",activation='softmax'))

In [19]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [20]:
model.fit_generator(x_train,steps_per_epoch=23,epochs=25,validation_data=x_test,validation_steps=10)

/tmp/wsuser/ipykernel_3684/2887656968.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,steps_per_epoch=23,epochs=25,validation_data=x_test,validation_steps=10)


Epoch 1/25
23/23 [==============================] - 34s 1s/step - loss: 1.3305 - accuracy: 0.3676 - val_loss: 1.2579 - val_accuracy: 0.4293
Epoch 2/25
23/23 [==============================] - 25s 1s/step - loss: 1.0391 - accuracy: 0.5380
Epoch 3/25
23/23 [==============================] - 26s 1s/step - loss: 0.8592 - accuracy: 0.6366
Epoch 4/25
23/23 [==============================] - 26s 1s/step - loss: 0.7849 - accuracy: 0.6648
Epoch 5/25
23/23 [==============================] - 24s 1s/step - loss: 0.7285 - accuracy: 0.7127
Epoch 6/25
23/23 [==============================] - 26s 1s/step - loss: 0.7398 - accuracy: 0.7028
Epoch 7/25
23/23 [==============================] - 26s 1s/step - loss: 0.6977 - accuracy: 0.7225
Epoch 8/25
23/23 [==============================] - 25s 1s/step - loss: 0.6371 - accuracy: 0.7465
Epoch 9/25
23/23 [==============================] - 26s 1s/step - loss: 0.5660 - accuracy: 0.7761
Epoch 10/25
23/23 [==============================] - 26s 1s/step - loss: 0.5

In [21]:
model.save("natural disaster.h5")

In [22]:
!pip install watson-machine-learning-client

In [23]:
from ibm_watson_machine_learning import APIClient


In [24]:
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"khN-3H_0VHXgMmugVGj2UajZXsXx7GiC5rjy7wC9HrH6"
}
client=APIClient(wml_credentials)

In [25]:
client

In [26]:
def guid_space_name(client,naturaldisaster_deploy):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==naturaldisaster_deploy)['metadata']['id'])

In [27]:
space_uid=guid_space_name(client,'naturaldisaster_deploy')
print("Space UID "+space_uid)

Space UID 2e012ebb-68e4-4f5e-8a6d-8241d726824b


In [28]:
client.set.default_space(space_uid)

'SUCCESS'

In [29]:
client.software_specifications.list(200)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

In [30]:
software_space_uid=client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')

In [31]:
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [32]:
ls

 dataset/   dataset.tgz  'natural disaster.h5'   naturaldisaster.tar.gb


In [33]:
!tar -zcvf  dataset.tgz 'digital natural disaster.h5' 

natural disaster.h5


In [34]:
model_details = client.repository.store_model(model="dataset.tgz",meta_props={
    client.repository.ModelMetaNames.NAME:"CNN modeln Building",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_space_uid
})

In [35]:
model_id=client.repository.get_model_id(model_details)

In [36]:
model_id

'0e6630ce-b3e6-48ad-b3c2-28dae0721748'

In [38]:
client.repository.download(model_id,'digital naturaldisaster.tar.gb')

Successfully saved model content to file: 'digital naturaldisaster.tar.gb'


'/home/wsuser/work/digital naturaldisaster.tar.gb'

In [39]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': 'acd9c798-6974-5d2f-a657-ce06e986df4d',
   'name': 'tensorflow_rt22.1-py3.9'},
  'type': 'tensorflow_2.7'},
 'metadata': {'created_at': '2022-11-19T14:58:25.191Z',
  'id': '0e6630ce-b3e6-48ad-b3c2-28dae0721748',
  'modified_at': '2022-11-19T14:58:32.730Z',
  'name': 'CNN modeln Building',
  'owner': 'IBMid-66100460UM',
  'resource_key': '8b1701d1-9f92-4cee-83b1-ab7d069079f3',
  'space_id': '2e012ebb-68e4-4f5e-8a6d-8241d726824b'},
 'system': {'warnings': []}}